In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')
sns.set()

In [ ]:
imdb = pd.read_csv("datasets/IMDB_Dataset.csv")
imdb.head()

In [ ]:
imdb['sentiment'].value_counts()

In [ ]:
text = imdb['review'][0]
print(text)
print("<=======================>")
print(word_tokenize(text))

In [ ]:
corpus = []
for text in imdb['review']:
    words = [word.lower() for word in word_tokenize(text)]
    corpus.append(words)

In [ ]:
num_words = len(corpus)
print(num_words)

In [ ]:
imdb.shape

In [ ]:
train_size = int(imdb.shape[0] * 0.8)
X_train = imdb['review'][:train_size]
y_train = imdb['sentiment'][:train_size]

X_test = imdb['review'][train_size:]
y_test = imdb['sentiment'][train_size:]

In [ ]:
tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=128, truncating='post', padding='post')

In [ ]:
X_train[0]

In [ ]:
len(X_train[0])

In [ ]:
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=128, truncating='post', padding='post')

In [ ]:
X_test[0]

In [ ]:
len(X_test[0])

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=num_words, output_dim=100,
                   input_length=128, trainable=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2, batch_size=32)